**Bono Trimestral**
-----
Archivo "mes string"\_bono_trimestral.xlsx

Asegurarse que en este archivo esten las columnas:
"DNI","LOGIN","GERENCIA1","GERENCIA2","ZONA","DEPARTAMENTO","POSICIÓN","NOMBRES","PORCENTAJE_TOTAL_PONDERADO","COMISIÓN","COMISION_ADICIONAL","PAGO","CAPTURA","CAPTURA 4.2","CAPTURA 4.3","GESTIÓN 1.3","Meses en el Puesto"

Si hay cambios en las columnsa modificar el script -> variable 'cols'

Primero verificar los puestos para los que aplica para hacer la modificación en el script -> variable 'filt1'

Ejecutar el script y luego verificar las formulas en el archivo 

TODO: En el archivo del calculo de reporte de productividad las condiciones logicas para ganar el bono convertirlas a macro con sentencias if - elseif para que sea mas facil de entender y modificar

In [2]:
import logging
import ecomis

#from Loader import fileloader_proto as fl
#from Loader import dfutils

import xlwings as xw
import pandas as pd
import numpy as np
import posixpath

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

"""
BONO TRIMESTRAL
Archivos de comisiones/secciones ini necesarios:
Comisionantes_GrandesCuentas_All
Comisionantes_Pymes_All

Puestos para los que aplica segun esquema actual:

Consultor Ventas Directas
Consultor Ventas Directas Senior
Consultores Captura Regiones
Ejecutivo Desarrollo de Negocio
Ejecutivo Desarrollo de Negocio Senior
Ejecutivo de Desarrollo Pyme Regiones
Ejecutivo de Regiones
Ejecutivo Remoto Regiones

Para bono trimestral importan las columnas de:
CAPTURA4.2 = resultado de VAS
CAPTURA4.3 = cumplimiento de VAS
GESTION1.3 = cumplimieento de CHURN
CAPTURA
"""

#Mes actual, a partir de esta cadena de texto se calculan las cadenas de los 2 meses anteriores
month = '202007'
date1_str = '202006' #mes anterior
date2_str = '202005' #mes anterior anterior

inifile = ecomis.ReadIniFile(mercado="empresas")

#Columnas de interes, que requerimos para las condiciones que impone el bono trimestral
cols = ["DNI","LOGIN","GERENCIA1","GERENCIA2","ZONA","DEPARTAMENTO","POSICIÓN","NOMBRES","PORCENTAJE_TOTAL_PONDERADO",\
        "COMISIÓN","COMISION_ADICIONAL","PAGO","CAPTURA.1","CAPTURA 4.2","CAPTURA 4.3","GESTIÓN 1.3","Meses en el Puesto"]

#Puestos para los cuales aplicar el bono trimestral
filt1 = ['CONSULTOR VENTAS DIRECTAS','CONSULTOR SENIOR VENTAS DIRECTAS','CONSULTOR DE NEGOCIOS REGIONES',\
         'EJECUTIVO DE DESARROLLO PYME','EJECUTIVO DE DESARROLLO PYME SENIOR','EJECUTIVO DE DESARROLLO PYME REGIONES',\
         'EJECUTIVO DE REGIONES','EJECUTIVO DE REGIONES REMOTO']
#Diccionario que especifica el renombre de columnas para que sea mas entendible
cols_rename = {'CAPTURA_4.2':'RESULTADO_VAS', 'CAPTURA_4.3':'CUMPL_VAS','GESTIÓN_1.3':'CUMPL_CHURN','CAPTURA.1':'CAPTURA'}

#Funcion para convertir un numero que representa un mes a una cadena, e.g: int(5) -> str('05')
def getMonthStringFromInt(mes_int):
  if mes_int < 10:
    mes_str = '0' + str(mes_int)
  else:
    mes_str = str(mes_int)
  return mes_str

#Calculamos las cadenas de texto que representan a los 2 meses anteriores
mes = int(month[-2:])
mes1 = mes - 1
mes2 = mes1 - 1
year = month[0:4]
mes1_str = getMonthStringFromInt(mes1)
date1_str = year + mes1_str
mes2_str = getMonthStringFromInt(mes2)
date2_str = year + mes2_str


### Mes Actual
# Cargamos los dataframes de Grandes Clientes y Pymes. Seleccionamos solo algunas columnas de interes (variable 'cols')
#section2 = ecomis.SectionObj(inifile,"Comisionantes_GrandesCuentas_All",month)
section2 = ecomis.SectionObj(inifile,"Comisionantes_All_Columns",month)
section2.setParameter('cols',cols)
section2.setParameter('allcols',0)
section2.setParameter('presetsheet',['Comisionantes'])
loader2 = ecomis.LoadFileProcess(section2)
result = loader2.loadFile()

# filtramos solo los puestos para los que aplica el bono trimestral.
# Además renombramos algunas columnas a nombres entendibles
stage = result["POSICIÓN"].isin(filt1)
result_filt = result[stage]
result_filt= result_filt.drop_duplicates('DNI') #Porsiacaso eliminamos duplicados segun DNI
result_filt.rename(columns=cols_rename, inplace=True)

# Tomamos la columna DNI que sera nuestra llave, ya que contiene los puestos para 
# los cuales aplica el Bono y los comisionantes del mes actual.
key_column = result_filt["DNI"]

### Mes anterior
# Cargamos los dataframes de Grandes Clientes y Pymes. Seleccionamos solo algunas columnas de interes (variable 'cols')
section_grandes = ecomis.SectionObj(inifile,"Comisionantes_All_Columns",date1_str)
section_grandes.setParameter('cols',cols)
section_grandes.setParameter('allcols',0)
section_grandes.setParameter('presetsheet',['Comisionantes'])
loader_grandes = ecomis.LoadFileProcess(section_grandes)
result = loader_grandes.loadFile()

#  filtramos solo los puestos para los que aplica el bono trimestral
result_filt_mes1 = pd.merge(key_column.to_frame(), result, on='DNI', how='left')
result_filt_mes1.rename(columns=cols_rename, inplace=True)
### End mes anterior anterior

### Mes anterior anterior
# Cargamos los dataframes de Grandes Clientes y Pymes. Seleccionamos solo algunas columnas de interes (variable 'cols')
section_grandes2 = ecomis.SectionObj(inifile,"Comisionantes_All_Columns",date2_str)
section_grandes2.setParameter('cols',cols)
section_grandes2.setParameter('allcols',0)
section_grandes2.setParameter('presetsheet',['Comisionantes'])
loader_grandes2 = ecomis.LoadFileProcess(section_grandes2)
result = loader_grandes2.loadFile()

#filtramos solo los puestos para los que aplica el bono trimestral
result_filt_mes2 = pd.merge(key_column.to_frame(), result, on='DNI', how='left')
result_filt_mes2.rename(columns=cols_rename, inplace=True)
### Mes anterior anterior


# Abrimos archivo de Bono Trimestral y escribimos los resultados
section4 = ecomis.SectionObj(inifile,"Bono_Trimestral",month)
bono_trimestral_filepath = section4.getParameter('filelist')[0]
wb = xw.Book(bono_trimestral_filepath)
mes0_sheet = wb.sheets("mes_actual")
mes1_sheet = wb.sheets("mes_ant")
mes2_sheet = wb.sheets("mes_ant_ant")

mes0_sheet.range('A2').options(index=False,header=False).value = result_filt
mes1_sheet.range('A2').options(index=False,header=False).value = result_filt_mes1
mes2_sheet.range('A2').options(index=False,header=False).value = result_filt_mes2

In [2]:
comisionantes_gc_df

,DNI,LOGIN,GERENCIA1,GERENCIA2,ZONA,DEPARTAMENTO,POSICIÓN,NOMBRES,PORCENTAJE_TOTAL_PONDERADO,COMISIÓN,COMISION_ADICIONAL,PAGO,CAPTURA.1,CAPTURA_4.2,CAPTURA_4.3,GESTIÓN_1.3,MESES_EN_EL_PUESTO
0,25720995,PDIAZ,GRANDES CUENTAS,CORPORACIONES,CORPORACIONES1,LIMA,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,PAUL ANTONIO DIAZ NOZIGLIA,1.5972,11550.00,0.0,11550.000000,1.210279,NaN,NaN,1.400000,2
1,40134270,AGOMEZDECORDOVA,GRANDES CUENTAS,CORPORACIONES,CORPORACIONES2,LIMA,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,ALVARO FERNANDO GOMEZ DE CORDOVA CALLIRGOS,1.4571,10890.00,0.0,10890.000000,1.491928,NaN,NaN,1.400000,147
2,40003514,RMESETH,GRANDES CUENTAS,CORPORACIONES,CORPORACIONES4,LIMA,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,ROMINA MESETH MACCHIAVELLO,1.4125,10890.00,0.0,11682.000000,1.553975,NaN,NaN,1.000000,204
3,10542531,FTOCCI,GRANDES CUENTAS,CORPORACIONES,CORPORACIONES3,LIMA,GERENTE DE NEGOCIOS CARTERA CORPORACIONES,FRANCISCO TOCCI VAN OORDT,1.0575,3795.00,0.0,11055.000000,1.054498,NaN,NaN,1.347751,28
4,0,FTOCCI-GOB,GRANDES CUENTAS,CORPORACIONES,CORPORACIONES3,LIMA,GERENTE DE NEGOCIOS CARTERA ADM. PÚBLICA,FRANCISCO TOCCI VAN OORDT - GOBIERNO,1.9571,7260.00,0.0,7260.000000,2.238734,1.838532e+06,3.342785,NaN,28
5,41812901,GMUNOZ,GRANDES CUENTAS,GRANDES CLIENTES,ARES,LIMA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,GERARDO DANIEL MUÑOZ JUGO,1.0271,7590.00,0.0,6600.000000,1.057560,1.179354e+04,1.209594,0.829173,28
6,41131333,EGOICOCHEA,GRANDES CUENTAS,GRANDES CLIENTES,CONDOR,LIMA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,EDUARDO GOICOCHEA ORTIZ DE ZEVALLOS,0.8998,5412.00,0.0,5412.000000,0.867029,8.074683e+03,0.828173,0.736368,189
7,42244365,JNARUSE,GRANDES CUENTAS,GRANDES CLIENTES,EAGLE,LIMA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,JORGE NARUSE BURGA,0.8962,5412.00,0.0,5412.000000,0.825019,6.072063e+03,0.622776,0.985873,33
8,40197601,SJIMENO,GRANDES CUENTAS,GRANDES CLIENTES,ESPARTA,LIMA,GERENTE DE NEGOCIOS CARTERA GRANDES CLIENTES,SANDRA JIMENO GOICOCHEA,1.1369,8448.00,0.0,8448.000000,1.082199,1.202431e+04,1.233263,1.130794,199
9,10867736,RREJAS,GRANDES CUENTAS,VENTA REGIONAL EMPRESA,REGIONAL CENTRO,REGIONAL CENTRO,GERENTE DE NEGOCIOS REGIONAL EMPRESAS,RODRIGO ALONSO REJAS COSTA,1.0955,6900.00,0.0,6900.000000,1.868194,9.682949e+03,0.709374,1.400000,15
